# Update ETF File

@2020/07/21 By Chunlei Zhou

In [1]:
import openpyxl as xl
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime

## Helper functions

In [2]:
def get_sheet_property(sheet):
    '''
    This function extract properties that will be used later on in the updating process.
    Input:  sheet: a worksheet that will be updated
    Output: nrow: how many rows are there in the worksheet before update
            ncol: how many columns are there in the worksheet before update
            starts: the date we will update the sheet from
    '''
    nrow = sheet.max_row
    ncol = sheet.max_column
    start_datetime = sheet.cell(row = nrow,column=1).value
    yyyy = start_datetime.strftime("%Y")
    mm = start_datetime.strftime("%m")
    dd = start_datetime.strftime("%d")
    qstart = yyyy + '-' + mm + '-' + dd
    spstart = mm + '-'+ dd + '-' + yyyy
    starts = {'sp500': spstart, 'quotes': qstart}
    return nrow, ncol, starts

def browser_quotes(executable_path):
    '''
    This function prepares the firefox browser
    '''
    browser = webdriver.Firefox(executable_path = executable_path)
    return browser

def adsblocker_Chrome(path_to_extension):
    '''
    This function builds the Chrome driver with the Ads Blocker (as an extension).
    If you do not want to block the adds, then do not call this fucntion, code your own driver instead.
    '''
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('load-extension=' + path_to_extension)
    browser = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
    return browser

def extract_sp_links(sheet):
    '''
    This function extracts all links from sheet %S&P500 and stores them in a dictionary
    '''
    splinks = {}
    for val in sheet.iter_cols(min_row = 5, min_col = 12, max_col = 20, max_row = 6, values_only = True):
        link = val[0]
        name = val[1]
        if name not in splinks.keys():
            splinks[name] = link
    return splinks

def extract_quotes_hist_link(key):
    '''
    This fucntion extract all links from yahoo finance for index in the sheet %Qoutes
    Input: key: the name of the ETF
    Output: link: direct to historical data page of the ETF
    '''
    print('Extracting', key, 'Information from website...')
    
    # Search the key using search box
    # Find location of the search box and put key into the box
    browser.find_element_by_xpath('//*[@id="yfin-usr-qry"]').send_keys(key)
    time.sleep(10)
    # Click search
    browser.find_element_by_xpath('//*[@id="header-desktop-search-button"]').click()
    
    # Extract Historical Data Link
    time.sleep(5)
    li = browser.find_element(By.XPATH, '//li[@data-test = "HISTORICAL_DATA"]')
    hist = li.find_element_by_xpath('.//a')
    link = hist.get_attribute('href')
    
    return link

def locate_quotes_table(link, start, end):
    '''
    This fucntion locate the table contains quotes data we want on the webpage.
    Input:  link: direct to historical data page of the ETF we are updating
            start: the date from which we update the ETF
            end: the date till which we update the ETF
    Output: tbody: a list contains all rows of the table we located
            thead: a list contains names of all columns of the table we located
    '''
    print('Locating target table on website...')
    # Open the web page
    browser.get(link)
    
    # Scroll web page to a suitable location
    selector = browser.find_element(By.XPATH, '//div[@class = "Pt(15px) drop-down-selector historical"]')
    apply = selector.find_element_by_xpath('.//button')
    apply.send_keys(Keys.DOWN)
    time.sleep(2)
    
    # Find the date manipulator
    date = selector.find_elements(By.XPATH, '//div[@data-test = "dropdown"]')
    date[2].click()
    time.sleep(2)
    
    # Set the start_date
    browser.find_element(By.XPATH, '//input[@name = "startDate"]').send_keys(start)
    time.sleep(2)
    
    # Set the end_date
    browser.find_element(By.XPATH, '//input[@name = "endDate"]').send_keys(end)
    time.sleep(2)
    
    # Click "Done" to submit all the settings
    date_menu = browser.find_element(By.XPATH, '//div[@data-test = "date-picker-menu"]')
    buttons = date_menu.find_elements_by_xpath('.//button')
    buttons[-2].click()
    time.sleep(2)
    
    # Find the dropdown menu to set what data to display:
    container = browser.find_elements(By.XPATH, '//div[@data-test = "select-container"]')
    container[0].click()
    time.sleep(2)
    # Show Historical Prices
    hist_menu = browser.find_element(By.XPATH, '//div[@data-test = "historicalFilter-menu"]')
    hist_menu.find_element(By.XPATH, '//div[@data-value = "history"]').click()
    time.sleep(2)
    
    # Find the dropdonw menu to set the data frequency
    container[1].click()
    freq_menu = browser.find_element(By.XPATH, '//div[@data-test = "historicalFrequency-menu"]')
    time.sleep(2)
    # Set Frequency as 'Daily'
    freq_menu.find_element(By.XPATH, '//div[@data-value = "1d"]').click()
    
    # Click 'Apply' button to submit all the settings
    apply.click()
    time.sleep(5)
    
    # Locate table
    table = browser.find_element(By.XPATH, '//table[@data-test = "historical-prices"]')
    time.sleep(2)
    
    # Scroll the web page to bottom
    html = browser.find_element_by_tag_name('html')
    html.send_keys(Keys.END)
    time.sleep(2)
    
    # Extract all rows into a list
    tbody = table.find_element_by_xpath('.//tbody')
    # Extract all heads into a list
    thead = table.find_element_by_xpath('.//thead')
    time.sleep(5)
    
    return tbody, thead

def locate_sp_table(key, link, spstart, spend):
    '''
    This function locates the table contains the S&P500 data on the web page
    Input:  key: the name of ETF
            link: direct to the historical data page of the ETF we are updating
            spstart: the date from when we are updating the ETF
            spend: the end date till when we are updating the ETF
    Output: tbody: a list contains all rows of the table we located
            thead: a list contains names of all columns of the table we located
    '''
    print('Extracting', key, 'Information from website...')
    
    # Load webpage
    browser.get(link)
    time.sleep(10)
    
    # Find the dropdown menu to select Historical Data for display
    time_frame = browser.find_element(By.XPATH,'//select[@id = "data_interval"]')
    time.sleep(2)
    # Scroll the page to a suitable position
    time_frame.send_keys(Keys.DOWN)
    time.sleep(2)
    # Select historical data
    browser.find_element(By.XPATH, './/select[@id = "data_interval"]/option[1]')
    time.sleep(2)
    # click and submit the selection
    time_frame.click()
    time.sleep(2)
    
    # Find data manipulator and set the updating data range
    browser.find_element(By.XPATH,'//div[@id = "widgetField"]').click()
    time.sleep(2)
    date_menu = browser.find_element(By.XPATH,'//div[@id = "ui-datepicker-div"]')
    time.sleep(2)
    # Set start date
    start_bar = date_menu.find_element(By.XPATH, '//input[@id = "startDate"]')
    start_bar.clear()
    time.sleep(2)
    start_bar.send_keys(spstart)
    time.sleep(2)
    # Set end date
    end_bar = date_menu.find_element(By.XPATH, '//input[@id = "endDate"]')
    end_bar.clear()
    time.sleep(2)
    end_bar.send_keys(spend)
    time.sleep(2)
    # Click 'Apply' button to submit all the settings
    apply = date_menu.find_element(By.XPATH, '//a[@id = "applyBtn"]')
    apply.click()
    
    # Locate table
    time.sleep(5)
    table = browser.find_element(By.XPATH,'//table[@id = "curr_table"]')
    
    # Scroll the web page to bottom
    html = browser.find_element_by_tag_name('html')
    html.send_keys(Keys.END)
    time.sleep(2)
    
    # Extract all heads from table to a list
    spthead = table.find_element_by_xpath('.//thead')
    # Extract all rows from table to a list
    sptbody = table.find_element_by_xpath('.//tbody')
    
    return sptbody, spthead

def get_index(thead, target, Date):
    '''
    This function finds the index of the data we use to update the sheet
    Input:  thead: a xpath of the table head
            target: the name of the column under which is the data we need
            Date: the name of the date column in the web page table
    Output: target_index: the index of the table column under which is the data we need
            date_indx: the index of the date column
    '''
    ths = thead.find_elements_by_xpath('.//th')
    heads = []
    # Extract all heads as strings from the table head row
    for th in ths:
        heads.append(th.text)
    # Obtain the index of date column
    date_index = heads.index(Date)
    # Obtain the index of column under which is the data we need
    target_index = heads.index(target)
    return target_index, date_index
    
def interpret_table(tbody, data_index, date_index):
    '''
    This function extract only the date data and the data we need from the whole table on web page
    Input:  tbody: a xpath of the table body
            data_index: the index of column under which is the data we need
            date_index: the index of the date column
    Output: asc_data[1:]: a list stores all the data we need to update the sheet from the next day of the start date till the end date
            asc_date[1:]: a list stores all the date corresponding to the date range we set
    '''
    print('Scraping information from target table...')
    
    # Extract all rows from table body
    trs = tbody.find_elements_by_xpath('.//tr')
    date = []
    data = []
    for tr in trs:
        tds = tr.find_elements_by_xpath('.//td')
        if data_index <= len(tds) - 1:
            # Find the data we need using index
            data.append(tds[data_index].text)
            # Find the corresponding date using index
            date.append(tds[date_index].text)
    # On the web page the data is from end to start, we want it to from start to end. Re-order data
    asc_data = data[::-1]
    ase_date = date[::-1]
    
    # The first data/date is the last row in the sheet. We only need data/date after it. Remove duplicate information
    return asc_data[1:], ase_date[1:]


def construct_dataframe(dic):
    '''
    This function transform the dictionary stores all information scraped from website to a data frame. 
    We need a data frame to further write the sheet we are updating.
    '''
    df = pd.DataFrame(dic)
    
    # Turn the data format of date to the one we want
    df['Date']= pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.date
    
    # Turn all string data to float data
    for col in df.columns[1:]:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            print('Cannot convert value in', col, 'to float. Save as string.')
    
    return df

def update_quotes_sheet(sheet, df):
    '''
    This fucntion write the sheet %Quotes using a data frame.
    '''
    for row in range(len(df)):
        sheet.append(df.iloc[row].tolist())

def update_sp500_sheet(sheet, df, nrow, L = 12, A = 1):
    '''
    This function write the sheet %S&P500 using a data frame.
    '''
    # Update columns L to T
    data = df.iloc[:,1:]
    for row in range(len(data)):
        values = data.iloc[row].tolist()
        for col, entry in enumerate(values, start = L):
            sheet.cell(row = nrow + row, column = col, value = entry)

    # Update column A
    dates = list(df.iloc[:,0])
    for row, entry in enumerate(dates, start = A):
        sheet.cell(row = nrow + row, column = A, value = entry)

## Set up global variables

In [3]:
# All quotes data are from yahoo finance
yahoo = 'https://finance.yahoo.com/' # Link of yahoo finance

#TODO: Have to change these paths to your own before you can run the code!
executable_path = '/Users/zhouchunlei/Downloads/geckodriver' # path of my firefox
path_to_extension = r'/Users/zhouchunlei/Desktop/JOB HUNTING/Internship/Week 1 7.17-7.23/4.15.0_0' # path of my Chrome Adblocker extension

# TODO: Have to change the file path to your own directory before you can run the code!
file = xl.load_workbook('01 ETF Portfolio-updated.xlsx') # Load the workbook
# Obtain the worksheets
sp500 = file['%S&P 500']
quotes = file['%QUOTES']

# Obtain properties for both sheets
# SP500
spnrow, spncol, spstarts = get_sheet_property(sheet = sp500)
spstart = spstarts['sp500']
# Quotes
qnrow, qncol, qstarts = get_sheet_property(sheet = quotes)
qstart = qstarts['quotes']

# TODO: Set your own end date before you run the code but do NOT change the format
qend = '2020-07-21' # Format for quotes end date: yyyy-mm-dd
spend = '07-21-2020'# Format for sp500 end date: mm-dd-yyy

# Set the column name as the input of function 'get_index(thead, target, Date)' for quotes and sp500 based on the web tables
q_setup = {'Date': 'Date', 'target': 'Close*'}
sp_setup = {'Date': 'Date', 'target': 'Price'}

# ETFs that no longer have data available
speckey = {'BDCL': 'https://finance.yahoo.com/quote/BDCL/history?p=BDCL', 
           'FINU': 'https://finance.yahoo.com/quote/FINU/history?p=FINU', 
           'LBDC': 'https://finance.yahoo.com/quote/LBDC/history?p=LBDC',
           'XLUY': 'https://finance.yahoo.com/quote/XLUY/history?p=XLUY',
           'OILU': 'https://finance.yahoo.com/quote/OILD/history?p=OILD',
           'GASX': 'https://finance.yahoo.com/quote/GASX/history?p=GASX',
           'MIDZ': 'https://finance.yahoo.com/quote/MIDZ/history?p=MIDZ',
           'RUSS': 'https://finance.yahoo.com/quote/RUSS/history?p=RUSS'}

## Main Function
### Quotes part

In [4]:
# Update Sheet Quotes
print('Updating Sheet Quotes...')
quotes_links = {} # stores links direct to historical data of quotes ETFs
q_dic = {} # stores all data scrapped from web pages for quotes ETFs

Updating Sheet Quotes...


In [12]:
browser = browser_quotes(executable_path) # open firefox driver
browser.get(yahoo) # open yahoo finance using firefox

# iterate for all ETFs in sheet %Quotes
for val in quotes.iter_cols(min_row = 2, min_col = 2, max_col = 164, max_row = 2, values_only = True):
    key = val[0] # get the name of ETF
    if key in speckey.keys():
        # Check for the data availability
        print('No new', key, 'data available. Extracting historical data for', key)
        link = speckey[key]
    else:
        link = extract_quotes_hist_link(key)
    
    # Store links
    if key not in quotes_links.keys():
        quotes_links[key] = link
    
    # Extract data and store in dictionary
    q_tbody, q_thead = locate_quotes_table(link, qstart, qend)
    q_data_index, q_date_index = get_index(q_thead, q_setup['target'], q_setup['Date'])
    q_data, q_date = interpret_table(q_tbody, q_data_index, q_date_index)
    q_dic['Date'] = q_date
    if key not in q_dic.keys():
        q_dic[key] = q_data
    time.sleep(10)

Extracting FXR Information from website...
Locating target table on website...
Scraping information from target table...
Extracting IYJ Information from website...
Locating target table on website...
Scraping information from target table...
Extracting VIS Information from website...
Locating target table on website...
Scraping information from target table...
Extracting PPA Information from website...
Locating target table on website...
Scraping information from target table...
Extracting CGW Information from website...
Locating target table on website...
Scraping information from target table...
Extracting FIDU Information from website...
Locating target table on website...
Scraping information from target table...
Extracting XLP Information from website...
Locating target table on website...
Scraping information from target table...
Extracting XLY Information from website...
Locating target table on website...
Scraping information from target table...
Extracting IBUY Information fro

#### Save links to a csv file. Run by demand

In [18]:
# Save all links to a csv. 
# If you want to save time in the future, change the code and use links directly loaded from this csv file.
# If you do not want to load links from csv and would rather to obtain links through web scraping, then do not tun this block.
with open('quotes_links.csv', 'w') as f:
    for key in quotes_links.keys():
        f.write("%s,%s\n"%(key,quotes_links[key]))

#### Save data frame to a csv file for safe. Run by demand

In [25]:
# Save all data for quotes to a csv file. Just in case. 
# If you do not want to save as a separate csv file, then do not run this block.
q_df = construct_dataframe(q_dic)
q_df.to_csv('quotes_' + qend +'.csv')

Cannot convert value in BDCL to float. Save as string.
Cannot convert value in FINU to float. Save as string.
Cannot convert value in LBDC to float. Save as string.
Cannot convert value in XLUY to float. Save as string.
Cannot convert value in GUSH to float. Save as string.
Cannot convert value in OILU to float. Save as string.
Cannot convert value in OILD to float. Save as string.
Cannot convert value in GASX to float. Save as string.
Cannot convert value in MIDZ to float. Save as string.
Cannot convert value in RUSS to float. Save as string.


### S&P500 Part

In [26]:
# Update Sheet SP500
print('Updating Sheet S&P_500...')
browser = adsblocker_Chrome(path_to_extension) # Open Google Chrome
splinks = extract_sp_links(sp500) # Extract all links from sheet %S&P500
browser.create_options() # Use adblocer for Chrome driver
sp_dic = {} # Dictionary stores all data scrapped from web page

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


Updating Sheet S&P_500...
 


[WDM] - Driver [/Users/zhouchunlei/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


In [28]:
# Iterate for ETFs from column L to T in sheet %S&P500
for key,link in splinks.items():
    # Extract data from web pages and store in dictionary
    sp_tbody, sp_thead = locate_sp_table(key, link, spstart, spend)
    sp_data_index, sp_date_index = get_index(sp_thead, sp_setup['target'], sp_setup['Date'])
    sp_data, sp_date = interpret_table(sp_tbody, sp_data_index, sp_date_index)
    sp_dic['Date'] = sp_date
    if key not in sp_dic.keys():
        sp_dic[key] = sp_data
    time.sleep(10)

Extracting S&P 500 REAL ESTATE Information from website...
Scraping information from target table...
Extracting S&P 500 COMMUNICATION SERVICES Information from website...
Scraping information from target table...
Extracting S&P 500 HEALTH CARE Information from website...
Scraping information from target table...
Extracting S&P 500 FINANCIALS Information from website...
Scraping information from target table...
Extracting S&P 500 ENERGY Information from website...
Scraping information from target table...
Extracting S&P 500 UTILITIES Information from website...
Scraping information from target table...
Extracting S&P 500 INDUSTRIALS Information from website...
Scraping information from target table...
Extracting S&P 500 CONSUMER STAPLES Information from website...
Scraping information from target table...
Extracting S&P 500 INFORMATION TECHNOLOGY Information from website...
Scraping information from target table...


#### Save links to a csv file. Run by demand

In [29]:
# Save all links to a csv. 
# If you want to change the code and use links directly loaded from this csv file, then run this block.
# If you do not want to load links from csv and would rather to extract links from the worksheet, then do not tun this block.
with open('sp500_links.csv', 'w') as f:
    for key in splinks.keys():
        f.write("%s,%s\n"%(key,splinks[key]))

#### Save data frame to a csv file for safe. Run by demand

In [30]:
# Save all data for quotes to a csv file. Just in case. 
# If you do not want to save as a separate csv file, then do not run this block.
sp_df = construct_dataframe(sp_dic)
sp_df.to_csv('sp500_' + spend + '.csv')

Cannot convert value in S&P 500 HEALTH CARE to float. Save as string.
Cannot convert value in S&P 500 INFORMATION TECHNOLOGY to float. Save as string.


### Update the workbook

In [31]:
# Update sheet %Quotes
update_quotes_sheet(quotes, q_df)

# Update sheet %S&P500
update_sp500_sheet(sp500, sp_df, spnrow, L = 12, A = 1)

#TODO: Change the save path to your own directory before you run the code
file.save('updated_' + spend + '.xlsx') #Save the updated file

print('Process Finished! The updated file is available to review in your directory.')

Process Finished! The updated file is available to review in your directory.
